# Stacking

# Setup

In [1]:
# notebook configuration
COLAB = False # will trigger manual installation of packages
USE_GPU = True 

In [2]:
# basic imports
from pathlib import Path
import os
import math
from datetime import datetime
import random

In [3]:
%matplotlib inline
%config Completer.use_jedi = False
os.environ['WANDB_NOTEBOOK_NAME'] = f"stacking_{datetime.now().strftime('%Y%m%d')}a.ipynb"

Now, non-stdlib imports

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# general ML tooling
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import log_loss, roc_auc_score
import wandb
from wandb.xgboost import wandb_callback
from wandb.lightgbm import wandb_callback
from sklearn.impute import SimpleImputer #, KNNImputer

import seaborn as sns

from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
# from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.preprocessing import RobustScaler#, StandardScaler #, MinMaxScaler, MaxAbsScaler, RobustScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
# from sklearn.feature_selection import SelectKBest, f_regression
from joblib import dump, load
# feature engineering tools
# from sklearn.feature_selection import mutual_info_regression
# import featuretools as ft

from BorutaShap import BorutaShap

In [5]:
from pytorch_widedeep import Trainer
from pytorch_widedeep.preprocessing import WidePreprocessor, TabPreprocessor
from pytorch_widedeep.models import Wide, TabMlp, WideDeep#, SAINT, TabTransformer, TabNet, TabFastFormer, TabResnet
from pytorch_widedeep.metrics import Accuracy
from torchmetrics import AUROC
import torch
from torch.optim import Adam, AdamW, Adagrad, SGD, RMSprop, LBFGS
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingWarmRestarts, CyclicLR, OneCycleLR, StepLR, CosineAnnealingLR
from pytorch_widedeep.callbacks import EarlyStopping, LRHistory, ModelCheckpoint

In [6]:
# import category_encoders as ce

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


Now, datapath setup

In [7]:
if COLAB:
    # mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    
    # handling datapath
    datapath = Path('/content/drive/MyDrive/kaggle/tabular_playgrounds/nov2021/')
    
else:
    # if on local machine
#     datapath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/sep2021/')  
    root = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/nov2021/')
    datapath = root/'datasets'
    # edapath = root/'EDA'
    # modelpath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/oct2021/models/')
    predpath = root/'preds'
    subpath = root/'submissions'
    
    for pth in [datapath, predpath, subpath]:
        pth.mkdir(exist_ok=True)
    


In [8]:
SEED = 42

# Function to seed everything
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

seed_everything(seed=SEED)

## Data Loading

In [9]:
# # dataset_params will initially include either trivial class instances or loaded, precomputed artifacts
# dataset_params = {
#     'train_source': str(datapath/'X-RobustScaled-pca_mle-umap_embedding_20211107-n_comp10-n_neighbors15-rs42.joblib'),
#     'target_source': str(datapath/'y_orig.joblib'),
#     'test_source': str(datapath/'X_test-RobustScaled-pca_mle-umap_embedding_20211107-n_comp10-n_neighbors15-rs42.joblib'),
#     'scaler': str(RobustScaler()),
#     'pca': str(load(datapath/'pca_mle-RobustScaled_orig_trainset.joblib')),
#     'umap': str(load(datapath/'umap_reducer-20211107-n_comp10-n_neighbors15-rs42-pca_mle-RobustScaled_orig_trainset.joblib')),
# }   

# # referring back to the already-entered attributes, specify how the pipeline was sequenced
# # dataset_params['preprocessing_pipeline'] = str([dataset_params['scaler'], dataset_params['pca'], dataset_params['umap']]) # ACTUALLY this is unwieldy
# dataset_params['preprocessing_pipeline'] = '[scaler, pca, umap]' # more fragile, but also more readable

dataset_params = load(datapath/'X_hybrid_20211108_meta.joblib')
dataset_params

{'train_source': ('/media/sf/easystore/kaggle_data/tabular_playgrounds/nov2021/datasets/X_hybrid_20211108.feather',
  ['/media/sf/easystore/kaggle_data/tabular_playgrounds/nov2021/datasets/X_orig.feather',
   '/media/sf/easystore/kaggle_data/tabular_playgrounds/nov2021/datasets/X-RobustScaled-pca_mle-umap_embedding_20211107-n_comp10-n_neighbors15-rs42.joblib']),
 'target_source': '/media/sf/easystore/kaggle_data/tabular_playgrounds/nov2021/datasets/y_orig.joblib',
 'test_source': ('/media/sf/easystore/kaggle_data/tabular_playgrounds/nov2021/datasets/X_test_hybrid_20211108.feather',
  ['/media/sf/easystore/kaggle_data/tabular_playgrounds/nov2021/datasets/X_test_orig-RobustScaled.feather',
   '/media/sf/easystore/kaggle_data/tabular_playgrounds/nov2021/datasets/X_test-RobustScaled-pca_mle-umap_embedding_20211107-n_comp10-n_neighbors15-rs42.joblib']),
 'scaler': ['RobustScaler()', 'RobustScaler()'],
 'pca': [None, "PCA(n_components='mle', random_state=42)"],
 'umap': [None,
  "UMAP(n_comp

In [14]:
# now, load the datasets and generate more metadata from them
X = pd.read_feather(Path(dataset_params['train_source'][0]))
y = load(Path(dataset_params['target_source']))
X_test = pd.read_feather(Path(dataset_params['test_source'][0]))

dataset_params['feature_count'] = X.shape[1]
dataset_params['instance_count'] = X.shape[0]
    

## Ex-Model Config

In [15]:
# meta-config for preprocessing and cross-validation, but NOT for model parameters
exmodel_config = {
#     "feature_selector": SelectKBest,
#     "k_best": 80,
#     "feature_selection_scoring": f_regression,
#     'random_state': SEED,
#     'feature_generation': ['NaN_counts', 'SummaryStats', 'NaN_OneHots'],
#     'subsample': 1,
    'cross_val_strategy': KFold, # None for holdout, or the relevant sklearn class
    'kfolds': 5, # if 1, that means just doing holdout
    'test_size': 0.2,
    **dataset_params
#     'features_created': False,
#     'feature_creator': None,
}

## Weights and Biases Run Config

Below is the configuration for a Weights and Biases (`wandb`) run.

In [12]:
# wandb config:
wandb_config = {
    'name': f"{os.environ['WANDB_NOTEBOOK_NAME'][:-6]}_{datetime.now().strftime('%H%M%S')}", # just removes the .ipynb extension, leaving the notebook filename's stem
    'tags': ['preprocessing'],
    'notes': "Running Big Three GBMs with default parameters and a hybrid dataset: a concatenation of 1. the original (RobustScaled) and 2. the original transformed with A. RobustScaler, B. PCA (MLE), 3. UMAP (n_neighbors=15, n_components=10)"
}

# Training

# Hyperparameters

In [13]:
# # optuna 20211004, thru 106 trials on unaltered original dataset
# best_xgboost_params = {
#     'n_estimators': 3878,
#     'max_depth': 4,
#     'learning_rate': 0.024785857161974977,
#     'reg_alpha': 26.867682044658245,
#     'reg_lambda': 10.839759074147148,
#     'subsample': 0.8208581489835881,
#     'min_child_weight': 8.829122644339664,
#     'colsample_bytree': 0.906420714280384,
#     'gamma': 1.472322916021486
# }

# # best as of 20211005, thru 65 trials on unaltered original dataset
# best_lightgbm_params = {
#     'n_estimators': 6631,
#     'max_depth': 10,
#     'learning_rate': 0.004677044539666842,
#     'reg_alpha': 19.334971246299116,
#     'reg_lambda': 0.024384251140153856,
#     'subsample': 0.5082183652689569,
#     'boosting_type': 'gbdt',
#     'min_child_samples': 9,
#     'num_leaves': 233,
#     'colsample_bytree': 0.5008014086989773
# }

# # catboost 20211001 on colab with 100 trials on GPU, unaltered original dataset
# best_catboost_params = {
#     'iterations': 29338,
#     'max_depth': 9,
#     'learning_rate': 0.004769831650275205,
#     'random_strength': 7,
#     'od_wait': 1968,
#     'reg_lambda': 28.435563240493586,
#     'border_count': 162,
#     'min_child_samples': 14,
#     'leaf_estimation_iterations': 1
# }

# # 20211021 lv2 on the K-Means 8-cluster, synth dataset
# lv2_xgboost_params = {
#     'n_estimators': 1534,
#     'max_depth': 4,
#     'learning_rate': 0.0062941159127744535,
#     'reg_alpha': 21.3946930650266,
#     'reg_lambda': 0.021003786013817635,
#     'subsample': 0.5726680367393964,
#     'min_child_weight': 0.07566661785187714,
#     'colsample_bytree': 0.7850419523745037,
#     'gamma': 4.26660233356059
# }

# # 20211021 lv2 on the K-Means 8-cluster, synth dataset
# lv2_lightgbm_params = {
#     'n_estimators': 5776,
#     'max_depth': 4,
#     'learning_rate': 0.0010172282832994653,
#     'reg_alpha': 0.013879765609402173,
#     'reg_lambda': 0.002787031048344079,
#     'subsample': 0.800000753298926,
#     'boosting_type': 'gbdt',
#     'min_child_samples': 11,
#     'num_leaves': 190,
#     'colsample_bytree': 0.9976443570341007
# }

# # 20211021 lv2 on the K-Means 8-cluster, synth dataset
# lv2_catboost_params = {
#     'iterations': 2000,
#     'depth': 6,
#     'learning_rate': 0.002984126581340097,
#     'random_strength': 0,
#     'od_wait': 334,
#     'reg_lambda': 33.469738674488084,
#     'border_count': 158,
#     'min_child_samples': 8,
#     'leaf_estimation_iterations': 4
# }

# # initial, non-default guess -- need to get optuna working (20211010)
# # basic_widedeep_tabmlp_params = {
    
# # }

# # basic_widedeep_trainer_params = {
# #     optimizers=AdamW()
# # }

In [14]:
from lightgbm.basic import LightGBMError

# Cross-Validator


In [15]:
def cross_validate_model(arch:str, X, y, X_test, params:dict={}, start_fold=0, 
                         exmodel_config=exmodel_config, wandb_config=wandb_config, 
                         random_state=42, shuffle_kfolds=True, wandb_tracked=True, encode_cats=False):
    """
    Function to handle model training process in the context of cross-validation -- via hold-out or via k-fold.
    If exmodel_config['cross_val_strategy'] == None, then any kfolds= input is ignored; otherwise, the number specified is used.
    
    :param kfolds: int specifying number of k-folds to use in cross-validation
    :param exmodel_config: dict containing general config including for cross-validation -- `kfold=1` implies hold-out
    """
#     if exmodel_config['kfolds'] == 1:
#         print("Proceeding with holdout")
#     X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
#                                                           test_size=0.2, 
#                                                           random_state=SEED)                 
    
    # prepare for k-fold cross-validation; random-state here is notebook-wide, not per-model
    # shuffle on the initial sets, but not subsequently -- performing the same operation twice means a very different dataset
    if shuffle_kfolds:
        kfold = exmodel_config['cross_val_strategy'](n_splits=exmodel_config['kfolds'], shuffle=True, random_state=SEED)
    else:
        kfold = exmodel_config['cross_val_strategy'](n_splits=exmodel_config['kfolds'], shuffle=False)
    
    if wandb_tracked:
        exmodel_config['arch'] = arch
        exmodel_config[f'{arch}_params'] = str(params)
        wandb.init(
            project="202111_Kaggle_tabular_playground",
            save_code=True,
            tags=wandb_config['tags'],
            name=wandb_config['name'],
            notes=wandb_config['notes'],
            config=exmodel_config
    )   
    
    # setup for serialization
    # runpath = Path(modelpath/f"{wandb_config['name']}_{arch}_{exmodel_config['kfolds']}folds/")
    # (runpath).mkdir(exist_ok=True)
    
    
    # initialize lists for out-of-fold preds and ground truth
    oof_preds, oof_y = [], []
    
    # initialize a numpy.ndarray containing the fold-model's preds for test set
    test_preds = np.zeros((X_test.shape[0]))
    
    # if using deep learning with pytorch-widedeep, do data preprocessing now, before splits
    if 'widedeep' in arch:
        # NOTE THAT ENCODING NOT DEPLOYED FOR THIS YET
        # preprocessing first
        wide_cols = [f for f in X.columns if X[f].nunique() == 2] #list(X_train.columns) if X_train.iloc[:,f].nunique() == 2] # binary indicator vars are wide
        cont_cols = [f for f in X.columns if X[f].nunique() > 2] #list(X_train.columns) if X_train.iloc[:,f].nunique() > 2] # others are cont

        # wide part
        # wide_preprocessor = WidePreprocessor(wide_cols=wide_cols)
        # X_wide = wide_preprocessor.fit_transform(X)
#         print(f"X_wide.shape = {X_wide.shape}")
#         X_wide = np.array(X_train[wide_cols])
        

        # deep part
        tab_preprocessor = TabPreprocessor(continuous_cols=cont_cols)#, embed_cols=embed_cols, )
        X_tab = tab_preprocessor.fit_transform(X)   
#         print(f"X_tab.shape = {X_tab.shape}")
        
        # transforming the test set
        X_test_wide = wide_preprocessor.transform(X_test)
        X_test_tab = tab_preprocessor.transform(X_test)
        
        # at this point, X_wide, X_tab, X_test_wide, and X_test_tab will all be np.ndarrays
    
#     else: # if using a GBM, simply convert the pd.DataFrames to np.ndarrays
#         X = np.array(X) # CAN YOU USE CATEGORY_ENCODERS ON NP.NDARRAYS?
#         X_test = np.array(X_test)
    
    for fold, (train_ids, valid_ids) in enumerate(kfold.split(X,y)):
#         print(f"type(train_ids) = {type(train_ids)} and train_ids.shape = {train_ids.shape}")
#         print(f"type(valid_ids) = {type(valid_ids)} and train_ids.shape = {valid_ids.shape}")
        if fold < start_fold: # skip folds that are already trained
            continue
        else:
            print(f"FOLD {fold}")
            print("---------------------------------------------------")
            y_train, y_valid = y[train_ids], y[valid_ids] # y will be an np.ndarray already; handling will be same regardless of model
            if 'widedeep' in arch: # handle wide and deep tabs in parallel
                X_train_wide, X_valid_wide = X_wide[train_ids, :], X_wide[valid_ids, :]
                X_train_tab, X_valid_tab = X_tab[train_ids, :], X_tab[valid_ids, :]
#                 print(f"X_train_wide.shape = {X_train_wide.shape}")
#                 print(f"X_train_tab.shape = {X_train_tab.shape}")
#                 print(f"X_test_wide.shape = {X_test_wide.shape}")
#                 print(f"X_test_tab.shape = {X_test_tab.shape}")
            else: # handle datasets for GBMs
                if isinstance(X, np.ndarray):
                    X_train, X_valid = X[train_ids], X[valid_ids]
                else:
                    X_train, X_valid = X.iloc[train_ids,:], X.iloc[valid_ids,:] # bc need pandas.DataFrames for ce
                # if encode_cats:
                #     encoder = ce.WOEEncoder(cols=categoricals)
                #     encoder.fit(X_train,y_train)
                #     X_train = encoder.transform(X_train)
                #     X_valid = encoder.transform(X_valid)
                # # exmodel_config['feature_count'] = len(X.columns)
                #     wandb.log({
                #         'feature_count': X_train.shape[1],
                #         'instance_count': X_train.shape[0],
                #         'encoder': str(encoder)
                #     })
#                 exmodel_config['instance_count'] = X_train.shape[0]
#                 exmodel_config['encoder'] = str(encoder)
#                     X_test = encoder.transform(X_test)
#                 y_train, y_valid = y[train_ids], y[valid_ids]
            
        # define models
        if arch == 'xgboost':
            model = XGBClassifier(
                booster='gbtree',
                tree_method='gpu_hist',
                random_state=random_state,
                n_jobs=-1, 
                verbosity=1, 
                objective='binary:logistic',
                **params)
            if wandb_tracked:
                model.fit(X_train, y_train, callbacks=[wandb.xgboost.wandb_callback()])
            else:
                model.fit(X_train, y_train)
            y_valid_preds = model.predict_proba(X_valid)[:,1]
            
            # add the fold-model's OOF preds and ground truths to the out-of-loop lists
            oof_preds.extend(y_valid_preds)
            oof_y.extend(y_valid)
            
            # add the fold's predictions to the model's test-set predictions (will divide later)
            test_preds += model.predict_proba(X_test)[:,1]


        elif arch == 'lightgbm':
            try:
                model = LGBMClassifier(
                    objective='binary',
                    random_state=random_state,
#                     device_type='cpu',
#                     n_jobs=-1,
    #                 eval_metric='auc',
                    device_type='gpu',
                    max_bin=63, # 15 might be even better for GPU perf, but depends on dataset -- see https://lightgbm.readthedocs.io/en/latest/GPU-Performance.html
                    gpu_use_dp=False, # forces use of single precision rather than double for better perf, esp on consumer Nvidia chips
                    **params)
                
                if wandb_tracked:
                    model.fit(X_train, y_train, callbacks=[wandb.lightgbm.wandb_callback()],)
                else:
                    model.fit(X_train, y_train)
            except LightGBMError:
                model = LGBMClassifier(
                    objective='binary',
                    random_state=random_state,
                    device_type='cpu',
                    n_jobs=-1,
    #                 eval_metric='auc',
    #                 device_type='gpu',
    #                 max_bin=63, # 15 might be even better for GPU perf, but depends on dataset -- see https://lightgbm.readthedocs.io/en/latest/GPU-Performance.html
    #                 gpu_use_dp=False, # forces use of single precision rather than double for better perf, esp on consumer Nvidia chips
                    **params)
                
                if wandb_tracked:
                    model.fit(X_train, y_train, callbacks=[wandb.lightgbm.wandb_callback()],)
                else:
                    model.fit(X_train, y_train)
            y_valid_preds = model.predict_proba(X_valid)[:,1]
            
            # add the fold-model's OOF preds and ground truths to the out-of-loop lists
            oof_preds.extend(y_valid_preds)
            oof_y.extend(y_valid)
            
            # add the fold's predictions to the model's test-set predictions (will divide later)
            test_preds += model.predict_proba(X_test)[:,1]

            
        elif arch == 'catboost':
            model = CatBoostClassifier(
                task_type='GPU',
                silent=True,
                random_state=random_state,
                **params) 
        
            model.fit(X_train, y_train)
            y_valid_preds = model.predict_proba(X_valid)[:,1]
            
            # add the fold-model's OOF preds and ground truths to the out-of-loop lists
            oof_preds.extend(y_valid_preds)
            oof_y.extend(y_valid)
            
            # add the fold's predictions to the model's test-set predictions (will divide later)
            test_preds += model.predict_proba(X_test)[:,1]
            
        elif 'widedeep' in arch: # only coding for TabMlp right now
#             X_train = pd.DataFrame(X_train, columns=[f"f{x}" for x in range(X_train.shape[1])])
#             X_valid = pd.DataFrame(X_valid, columns=[f"f{x}" for x in range(X_valid.shape[1])])
#             X_test = pd.DataFrame(X_test, columns=[f"f{x}" for x in range(X_test.shape[1])])
            
            wide = Wide(wide_dim=np.unique(X_wide).shape[0], pred_dim=1)
            deeptabular = TabMlp(
                mlp_hidden_dims=[64,32],
                column_idx=tab_preprocessor.column_idx,
            #     embed_input=tab_preprocessor.embeddings_input,
                continuous_cols=cont_cols,
            )
            
            # model instantiation and training
            model = WideDeep(wide=wide, deeptabular=deeptabular)
            
            
            n_epochs = 300

            # pytorch hyperparams
            wide_opt = AdamW(model.wide.parameters(), lr=0.1)
            deep_opt = AdamW(model.deeptabular.parameters(), lr=0.1)
            
            wide_sch = OneCycleLR(optimizer=wide_opt, max_lr=0.01, steps_per_epoch=X_train_wide.shape[0], epochs=n_epochs)
            deep_sch = OneCycleLR(optimizer=deep_opt, max_lr=0.01, steps_per_epoch=X_train_tab.shape[0], epochs=n_epochs)
            
            optimizers = {'wide': wide_opt, 'deeptabular': deep_opt }
            lr_schedulers = {'wide': wide_sch, 'deeptabular': deep_sch }
            
            
            callbacks = [
                LRHistory(n_epochs=n_epochs), 
            ]
            
            # trainer
            trainer = Trainer(model=model, 
                              objective='binary', 
                              metrics=[Accuracy], # with AUROC got TypeError: '>' not supported between instances of 'NoneType' and 'int' 
                              seed=random_state, 
                              optimizers=optimizers,
                              callbacks=callbacks
                             )
            
#             print(f"type(X_train_wide) is {type(X_train_wide)} and type(X_train_tab) is {type(X_train_tab)}")
            trainer.fit( # this is where problem is beginning
                X_wide=X_train_wide,
                X_tab=X_train_tab,
                target=y_train,
                n_epochs=n_epochs,
                batch_size=1024, # default value is 32
#                 val_split=0.2, # no need for this
            )
            
            y_valid_preds = trainer.predict_proba(X_wide=X_valid_wide, X_tab=X_valid_tab, batch_size=1024)[:,1]
            
            # add the fold-model's OOF preds and ground truths to the out-of-loop lists
            oof_preds.extend(y_valid_preds)
            oof_y.extend(y_valid)
            
            
            # test set inference
            fold_test_preds = trainer.predict_proba(X_wide=X_test_wide, X_tab=X_test_tab, batch_size=1024)[:,1]
            test_preds += fold_test_preds
            

        
        
       
        
        

#         valid_loss = log_loss(y_valid, y_pred)
        # give the valid AUC score, for edification
        fold_valid_auc = roc_auc_score(y_valid, y_valid_preds)
        if wandb_tracked:
            wandb.log({f'fold{fold}_valid_roc_auc': fold_valid_auc})
        print(f"Valid AUC for fold {fold} is {fold_valid_auc}")   
        # dump(model, Path(runpath/f"{arch}_fold{fold}_rs{random_state}_model.joblib"))

    model_valid_auc = roc_auc_score(oof_y, oof_preds)
    print(f"Valid AUC score for {arch} model is {model_valid_auc}")
    if wandb_tracked:
        wandb.log({'overall_valid_auc': model_valid_auc,
                   'model_params': str(model.parameters()) if 'widedeep' in arch else str(model.get_params()),
                   'model_seed': random_state,
                  })
        wandb.finish()
    
    # finalize test preds
    test_preds /= exmodel_config['kfolds']
    
    # save OOF preds and test-set preds
#     if 'widedeep' in arch:
#         dump(oof_preds, Path(predpath/f"{wandb_config['name']}_{arch}_{exmodel_config['kfolds']}folds_{n_epochs}epochs-per-fold_rs{random_state}_oof_preds.joblib"))
#         dump(test_preds, Path(predpath/f"{wandb_config['name']}_{arch}_{exmodel_config['kfolds']}folds_{n_epochs}epochs-per-fold_rs{random_state}_test_preds.joblib"))
    
#     else:
#         dump(oof_preds, Path(predpath/f"{wandb_config['name']}_{arch}_{exmodel_config['kfolds']}folds_rs{random_state}_oof_preds.joblib"))
#         dump(test_preds, Path(predpath/f"{wandb_config['name']}_{arch}_{exmodel_config['kfolds']}folds_rs{random_state}_test_preds.joblib"))
    
    if not (datapath/f"{exmodel_config['kfolds']}folds_rs{SEED}_oof_y.joblib").is_file():
        dump(oof_y, predpath/f"{exmodel_config['kfolds']}folds_rs{SEED}_oof_y.joblib")
    
#     if wandb_tracked:
# #         if 'widedeep' in arch:
#         wandb.log({'model_valid_auc': model_valid_auc,
# #                    'oof_preds': oof_preds,
# #                    'test_preds': test_preds,
#                    'model_params': str(model.parameters()) if 'widedeep' in arch else str(model.get_params()), 
#         #                    'model_params': str(model.get_params()),
#         })
# #         wandb.log({'model_valid_auc': model_valid_auc,
# #                    'oof_preds': oof_preds,
# #                    'test_preds': test_preds,
# # #                    'model_params': str(model.get_params()),
# #                   })
#         wandb.finish()
    return oof_preds, test_preds
        

# Predictions

## Level One

I'm going to try adding more models to this run -- a third seed for everything -- and also train the TabMLP models deeper -- 300 epochs/fold. I'm not confident that I've found a set of predictions that use the best model hyperparameters on the original dataset, so I'm going to just run this all over again. In the future, I **need** to do better at tracking artifacts.

In [16]:
architectures = ['xgboost', 'lightgbm', 'catboost']#, 'widedeep-tabmlp', 'widedeep-saint']

In [17]:
model_seeds = [42]#, 1983, 550, 1701, 2063]

In [18]:
# lv1_params = {
#     'xgboost': { # thru trial 71 cross-validated on RobustScaled orig dataset, as of 202111031537
#         'n_estimators': 8784,
#         'depth': 9,
#         'learning_rate': 0.004167178645277267,
#         'reg_alpha': 0.007249923752866805,
#         'reg_lambda': 0.08945255185214125,
#         'subsample': 0.7288417897178108,
#         'min_child_weight': 3.9187138542139577,
#         'colsample_bytree': 0.5284325948533055,
#         'gamma': 3.0265775282730822}
#     'lightgbm': { # thru trial 38 cross-validated on RobustScaled orig dataset, as of 202111031440
#         'n_estimators': 5108,
#         'max_depth': 4,
#         'learning_rate': 0.01253791570387513,
#         'reg_alpha': 0.015194423057424834,
#         'reg_lambda': 10.289397982794664,
#         'subsample': 0.996318668039871,
#         'boosting_type': 'goss',
#         'min_child_samples': 18,
#         'num_leaves': 218,
#         'colsample_bytree': 0.580388444330496 },
#     'catboost': { # thur trial 45 cross-validated on RobustScaled orig dataset, as of 202111041011
#         'iterations': 29222,
#         'depth': 7,
#         'learning_rate': 0.0067277390824230605,
#         'random_strength': 1,
#         'od_wait': 1989,
#         'reg_lambda': 51.436909447809484,
#         'border_count': 239,
#         'min_child_samples': 11,
#         'leaf_estimation_iterations': 2}
#     # 'widedeep-tabmlp': #todo,
#     # 'widedeep-saint': #todo,
# }

In [19]:
oof_lv1, test_lv1 = pd.DataFrame(), pd.DataFrame() # initialize dataframes

### Generating

In [20]:
for arch in architectures:
    for model_seed in model_seeds:
        # update exmodel_config here
        oof_pred, test_pred = cross_validate_model(arch=arch, X=X, y=y, X_test=X_test, 
                                         wandb_config=wandb_config,
                                         random_state=model_seed,
                                         # params=lv1_params[arch],
                                         exmodel_config=exmodel_config, 
                                         wandb_tracked=True
                                        )
        oof_lv1[f'{arch}{model_seed}'] = oof_pred
        test_lv1[f'{arch}{model_seed}'] = test_pred

wandb: Currently logged in as: hushifang (use `wandb login --relogin` to force relogin)


FOLD 0
---------------------------------------------------


The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
Old style callback is deprecated.  See: https://xgboost.readthedocs.io/en/latest/python/callbacks.html


Valid AUC for fold 0 is 0.5644406380027869
FOLD 1
---------------------------------------------------


The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
Old style callback is deprecated.  See: https://xgboost.readthedocs.io/en/latest/python/callbacks.html


...

fold0_valid_roc_auc,▁
fold1_valid_roc_auc,▁
fold2_valid_roc_auc,▁
fold3_valid_roc_auc,▁
fold4_valid_roc_auc,▁
model_seed,▁
overall_valid_auc,▁
fold0_valid_roc_auc,0.56444
fold1_valid_roc_auc,0.56342
fold2_valid_roc_auc,0.56355
fold3_valid_roc_auc,0.56152


FOLD 0
---------------------------------------------------
Valid AUC for fold 0 is 0.5672587982378053
FOLD 1
---------------------------------------------------
Valid AUC for fold 1 is 0.567947705561272
FOLD 2
---------------------------------------------------
Valid AUC for fold 2 is 0.5663871365611621
FOLD 3
---------------------------------------------------
Valid AUC for fold 3 is 0.5662054451105053
FOLD 4
---------------------------------------------------
Valid AUC for fold 4 is 0.5670741242747432
Valid AUC score for lightgbm model is 0.5669384738085824


fold0_valid_roc_auc,▁
fold1_valid_roc_auc,▁
fold2_valid_roc_auc,▁
fold3_valid_roc_auc,▁
fold4_valid_roc_auc,▁
model_seed,▁
overall_valid_auc,▁
fold0_valid_roc_auc,0.56726
fold1_valid_roc_auc,0.56795
fold2_valid_roc_auc,0.56639
fold3_valid_roc_auc,0.56621


FOLD 0
---------------------------------------------------
Valid AUC for fold 0 is 0.5683869449387695
FOLD 1
---------------------------------------------------
Valid AUC for fold 1 is 0.5687585261401885
FOLD 2
---------------------------------------------------
Valid AUC for fold 2 is 0.5677744774690742
FOLD 3
---------------------------------------------------
Valid AUC for fold 3 is 0.5676115365484723
FOLD 4
---------------------------------------------------
Valid AUC for fold 4 is 0.5679779281924575
Valid AUC score for catboost model is 0.5680755819984286


fold0_valid_roc_auc,▁
fold1_valid_roc_auc,▁
fold2_valid_roc_auc,▁
fold3_valid_roc_auc,▁
fold4_valid_roc_auc,▁
model_seed,▁
overall_valid_auc,▁
fold0_valid_roc_auc,0.56839
fold1_valid_roc_auc,0.56876
fold2_valid_roc_auc,0.56777
fold3_valid_roc_auc,0.56761


In [23]:
X_test.head()

,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99
0,600000,0.003229,4.838660,585.529,2.282910,0.713180,3.907830,0.480696,1.482270,4.891810,...,0.112910,1.073550,0.122149,0.086330,0.036010,0.010619,0.290343,1.898200,0.131533,0.012047
1,600001,0.008602,0.505536,-100.099,3.012670,0.027199,1.194610,5.036620,2.517440,4.553890,...,-0.020214,2.622340,0.123307,0.033063,0.123059,0.005771,-0.392923,3.689640,0.047418,0.120015
2,600002,1.461000,2.437260,-112.964,3.541230,0.752338,4.338310,1.648080,4.699910,1.950250,...,-0.011036,2.030180,-0.000426,0.084091,0.123605,0.499554,4.054650,3.330670,0.108843,0.064687
3,600003,0.140556,3.085610,179.451,0.573945,0.057342,2.216790,1.623480,0.526174,1.542540,...,0.050117,0.221613,0.045298,0.129966,0.004015,0.018279,2.696580,-0.533491,0.052524,0.011058
4,600004,0.128876,5.199760,107.466,-0.497149,0.080220,0.458121,0.629839,5.240460,-0.232279,...,0.058860,2.660430,0.135425,0.036481,0.093912,0.056315,1.110710,3.584470,0.145319,-0.050393


In [24]:
X_test = X_test.drop('id', axis=1)

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


In [ ]:
# oof_lv1.to_feather(predpath/f'stacking_oof.feather')
# test_lv1.to_feather(predpath/f'stacking_test.feather')

### Serialization

In [20]:
# oof_lv1 = pd.read_feather(predpath/f"stacking_manual_20211027_143755nb-20211028183233run-X_orig-oof-lv1.feather")#, columns=[str(x) for x in range()])
# test_lv1 = pd.read_feather(predpath/f"stacking_manual_20211027_143755nb-20211028183233run-X_orig-test-lv1.feather")
oof_y = load(predpath/f"{exmodel_config['kfolds']}folds_rs{SEED}_oof_y.joblib")

In [21]:
# oof_lv1_1701 = pd.read_feather(altdatapath/'oof_lv1_1701_models.feather')
# test_lv1_1701 = pd.read_feather(altdatapath/'test_lv1_1701_models.feather')
# oof_lv1_2063 = pd.read_feather(altdatapath/'oof_lv1_2063_models.feather')
# test_lv1_2063 = pd.read_feather(altdatapath/'test_lv1_2063_models.feather')

In [22]:
# oof_lv1.iloc[:10,:]

In [23]:
# oof_lv1_1701.head()

In [24]:
# oof_lv1_2063.head()

In [25]:
# oof_lv1.join(oof_lv1_1701).join(oof_lv1_2063)

In [26]:
# test_lv1.join(test_lv1_1701).join(test_lv1_2063)

In [89]:
oof_lv1.to_feather(altdatapath/'oof_lv1_5rs.feather')
# oof_lv1 = pd.read_feather(altdatapath/'oof_lv1_5rs.feather')

In [90]:
test_lv1.to_feather(altdatapath/'test_lv1_5rs.feather')
# test_lv1 = pd.read_feather(altdatapath/'test_lv1_5rs.feather')

In [29]:
oof_lv1.head()

,lgb42,lgb1983,lgb550,xgb42,xgb1983,xgb550,cat42,cat1983,cat550,tabmlp42,tabmlp1983,tabmlp550
0,0.643063,0.644761,0.651015,0.627152,0.666962,0.658337,0.633626,0.637863,0.618124,0.528318,0.551580,0.455730
1,0.979158,0.980269,0.979628,0.975591,0.976313,0.975725,0.978221,0.978872,0.979702,0.960835,0.958873,0.965270
2,0.694800,0.735218,0.685842,0.674095,0.670068,0.668585,0.758380,0.695742,0.687652,0.722547,0.600854,0.755067
3,0.315001,0.310552,0.317150,0.294775,0.321891,0.326354,0.277934,0.281502,0.280313,0.200837,0.214472,0.273882
4,0.086256,0.089822,0.081587,0.069561,0.071851,0.072671,0.060639,0.061077,0.068461,0.060948,0.034382,0.082108


In [30]:
oof_lv1 = oof_lv1.join(pd.read_feather(altdatapath/'oof_lv1_1701_models.feather')).join(pd.read_feather(altdatapath/'oof_lv1_2063_models.feather'))

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


In [31]:
oof_lv1.head()

,lgb42,lgb1983,lgb550,xgb42,xgb1983,xgb550,cat42,cat1983,cat550,tabmlp42,tabmlp1983,tabmlp550,lgb1701,xgb1701,cat1701,tablmp1701,lgb2063,xgb2063,cat2063,tablmp2063
0,0.643063,0.644761,0.651015,0.627152,0.666962,0.658337,0.633626,0.637863,0.618124,0.528318,0.551580,0.455730,0.647845,0.636991,0.604513,0.550344,0.636336,0.652631,0.630627,0.550438
1,0.979158,0.980269,0.979628,0.975591,0.976313,0.975725,0.978221,0.978872,0.979702,0.960835,0.958873,0.965270,0.979586,0.975426,0.977638,0.967344,0.979467,0.973305,0.977416,0.966318
2,0.694800,0.735218,0.685842,0.674095,0.670068,0.668585,0.758380,0.695742,0.687652,0.722547,0.600854,0.755067,0.716650,0.679206,0.788293,0.634997,0.642301,0.663404,0.696574,0.747513
3,0.315001,0.310552,0.317150,0.294775,0.321891,0.326354,0.277934,0.281502,0.280313,0.200837,0.214472,0.273882,0.301760,0.318009,0.272723,0.223079,0.311523,0.316786,0.280914,0.174859
4,0.086256,0.089822,0.081587,0.069561,0.071851,0.072671,0.060639,0.061077,0.068461,0.060948,0.034382,0.082108,0.094915,0.065146,0.066420,0.050387,0.084422,0.068258,0.064579,0.073367


In [32]:
test_lv1 = test_lv1.join(pd.read_feather(altdatapath/'test_lv1_1701_models.feather')).join(pd.read_feather(altdatapath/'test_lv1_2063_models.feather'))

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


In [33]:
oof_lv1

,lgb42,lgb1983,lgb550,xgb42,xgb1983,xgb550,cat42,cat1983,cat550,tabmlp42,tabmlp1983,tabmlp550,lgb1701,xgb1701,cat1701,tablmp1701,lgb2063,xgb2063,cat2063,tablmp2063
0,0.643063,0.644761,0.651015,0.627152,0.666962,0.658337,0.633626,0.637863,0.618124,0.528318,0.551580,0.455730,0.647845,0.636991,0.604513,0.550344,0.636336,0.652631,0.630627,0.550438
1,0.979158,0.980269,0.979628,0.975591,0.976313,0.975725,0.978221,0.978872,0.979702,0.960835,0.958873,0.965270,0.979586,0.975426,0.977638,0.967344,0.979467,0.973305,0.977416,0.966318
2,0.694800,0.735218,0.685842,0.674095,0.670068,0.668585,0.758380,0.695742,0.687652,0.722547,0.600854,0.755067,0.716650,0.679206,0.788293,0.634997,0.642301,0.663404,0.696574,0.747513
3,0.315001,0.310552,0.317150,0.294775,0.321891,0.326354,0.277934,0.281502,0.280313,0.200837,0.214472,0.273882,0.301760,0.318009,0.272723,0.223079,0.311523,0.316786,0.280914,0.174859
4,0.086256,0.089822,0.081587,0.069561,0.071851,0.072671,0.060639,0.061077,0.068461,0.060948,0.034382,0.082108,0.094915,0.065146,0.066420,0.050387,0.084422,0.068258,0.064579,0.073367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,0.877003,0.866812,0.859368,0.873962,0.876338,0.882213,0.876955,0.876530,0.873538,0.872165,0.898925,0.846161,0.876102,0.867594,0.875290,0.880338,0.863979,0.866435,0.880339,0.894021
999996,0.735823,0.732218,0.702689,0.777820,0.785770,0.789151,0.766000,0.766401,0.768164,0.751403,0.660141,0.760324,0.758805,0.749080,0.768249,0.743479,0.712077,0.768061,0.726242,0.661662
999997,0.151834,0.155176,0.146293,0.140220,0.146466,0.153901,0.164341,0.165027,0.164354,0.103621,0.129194,0.094390,0.151669,0.151052,0.155469,0.077828,0.143994,0.135747,0.161125,0.168268
999998,0.205738,0.182986,0.196857,0.216337,0.200384,0.197216,0.232795,0.227559,0.229330,0.277918,0.401489,0.343227,0.222530,0.217675,0.229789,0.335021,0.186798,0.224917,0.226820,0.178284


Now, I'll quickly do some Optuna for the above lv1 tables.

## Level Two

In [34]:
# oof_lv1 = pd.read_feather(predpath/'stacking_manual_20211020_104938_X_orig+KMeans8+synth_oof_lv1.feather')
# oof_lv1 = pd.read_feather(predpath/'stacking_manual_20211011_092728_oof_lv1.feather') # basis for best original dataset run as of 20211021
# test_lv1 = pd.read_feather(predpath/'stacking_manual_20211020_104938_X_orig+KMeans8+synth_test_lv1.feather')
# test_lv1 = pd.read_feather(predpath/'stacking_manual_20211011_092728_test_lv1.feather') # basis for best original dataset run as of 20211021
# oof_y = load(predpath/'5folds_rs42_oof_y.joblib')

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


In [91]:
lv2_xgboost_params = {
    'n_estimators': 8398,
    'max_depth': 7,
    'learning_rate': 0.005152495267590912,
    'reg_alpha': 0.003501877866082655,
    'reg_lambda': 0.5892650659577978,
    'subsample': 0.5321195352481216,
    'min_child_weight': 1.7950489828988663,
    'colsample_bytree': 0.6100580666401978,
    'gamma': 7.836182280294878}

# trial 27
lv2_lightgbm_params = {
    'n_estimators': 5978,
    'max_depth': 4,
    'learning_rate': 0.002972483637079397,
    'reg_alpha': 0.0066240595682091315,
    'reg_lambda': 0.023021277110080198,
    'subsample': 0.762378215816119,
    'boosting_type': 'goss',
    'min_child_samples': 14,
    'num_leaves': 108,
    'colsample_bytree': 0.6831809216468459}

# trial 62
lv2_catboost_params = {
    'iterations': 2633, 
    'depth': 10, 
    'learning_rate': 0.0023597486442471613, 
    'random_strength': 2, 
    'od_wait': 1597, 
    'reg_lambda': 60.949736178635824, 
    'border_count': 109, 
    'min_child_samples': 20, 
    'leaf_estimation_iterations': 3
}

In [92]:
oof_lv2, test_lv2 = pd.DataFrame(), pd.DataFrame()

In [93]:
oof_y = pd.Series(oof_y)

In [94]:
oof_lv2_xgb42, test_lv2_xgb42 = cross_validate_model(library='xgboost', X=oof_lv1, y=oof_y, X_test=test_lv1, 
                                         wandb_config=wandb_config,
                                         random_state=42,
                                         params=lv2_xgboost_params,
                                         exmodel_config=exmodel_config, 
                                         shuffle_kfolds=False,
                                         wandb_tracked=False,
                                         encode_cats=False
                                        )

dump(oof_lv2_xgb42, predpath/f"{wandb_config['name']}nb-{datetime.now().strftime('%Y%m%d%H%M%S')}run-X_orig_oof_lv2_xgboost42_preds.joblib")
dump(test_lv2_xgb42, predpath/f"{wandb_config['name']}nb-{datetime.now().strftime('%Y%m%d%H%M%S')}run-X_orig_test_lv2_xgboost42_preds.joblib")



FOLD 0
---------------------------------------------------


The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[16:55:20] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Valid AUC for fold 0 is 0.8578150664470746
FOLD 1
---------------------------------------------------


The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[16:55:57] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Valid AUC for fold 1 is 0.8566466212062318
FOLD 2
---------------------------------------------------


...

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[16:57:09] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Valid AUC for fold 3 is 0.8564849475361453
FOLD 4
---------------------------------------------------


The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[16:57:43] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Valid AUC for fold 4 is 0.8572123879070284
Valid AUC score for xgboost model is 0.8571320078395062


['/home/sf/code/kaggle/tabular_playgrounds/oct2021/preds/stacking_manual_20211031_160241nb-20211031165842run-X_orig_test_lv2_xgboost42_preds.joblib']

In [95]:
# oof_xgb_f0_rs1983 = load('/media/sf/easystore/kaggle_data/tabular_playgrounds/sep2021/models/stacking_manual_20210926_211701_xgboost_5folds/xgboost_fold0_model.joblib')
# oof_xgb_f0_rs42 = load('/media/sf/easystore/kaggle_data/tabular_playgrounds/sep2021/models/stacking_manual_20210925_212129_xgboost_5folds/xgboost_fold0_model.joblib')

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


In [96]:
oof_lv2_cat42, test_lv2_cat42 = cross_validate_model(library='catboost', X=oof_lv1, y=oof_y, X_test=test_lv1, 
                                         wandb_config=wandb_config,
                                         random_state=42,
                                         params=lv2_catboost_params,
                                         exmodel_config=exmodel_config, 
                                         shuffle_kfolds=False,
                                         wandb_tracked=False,
                                         encode_cats=False
                                        )

FOLD 0
---------------------------------------------------


KeyboardInterrupt: 

In [ ]:
dump(oof_lv2_cat42, predpath/f"{wandb_config['name']}nb-{datetime.now().strftime('%Y%m%d%H%M%S')}run-X_orig_oof_lv2_catboost42_preds.joblib")
dump(test_lv2_cat42, predpath/f"{wandb_config['name']}nb-{datetime.now().strftime('%Y%m%d%H%M%S')}run-X_orig_test_lv2_catboost42_preds.joblib")

In [ ]:

oof_lv2_lgb42, test_lv2_lgb42 = cross_validate_model(library='lightgbm', X=oof_lv1, y=oof_y, X_test=test_lv1, 
                                         wandb_config=wandb_config,
                                         random_state=42,
                                         params=lv2_lightgbm_params,
                                         exmodel_config=exmodel_config,
                                         shuffle_kfolds=False,
                                         wandb_tracked=False,
                                         encode_cats=False
                                        )

In [ ]:
dump(oof_lv2_lgb42, predpath/f"{wandb_config['name']}nb-{datetime.now().strftime('%Y%m%d%H%M%S')}run-X_orig_oof_lv2_lightgbm42_preds.joblib")
dump(test_lv2_lgb42, predpath/f"{wandb_config['name']}nb-{datetime.now().strftime('%Y%m%d%H%M%S')}run-X_orig_test_lv2_lightgbm42_preds.joblib")

In [97]:
oof_lv2['xgboost'] = oof_lv2_xgb42
oof_lv2['catboost'] = oof_lv2_cat42
oof_lv2['lightgbm'] = oof_lv2_lgb42

test_lv2['xgboost'] = test_lv2_xgb42
test_lv2['catboost'] = test_lv2_cat42
test_lv2['lightgbm'] = test_lv2_lgb42

In [98]:
# oof_lv1_df = pd.read_feather(predpath/f"{wandb_config['name']}_oof_lv1.feather)

In [99]:
oof_lv2_full = oof_lv2.join(oof_lv1)

In [100]:
oof_lv2_full.head()

,xgboost,catboost,lightgbm,lgb42,lgb1983,lgb550,xgb42,xgb1983,xgb550,cat42,...,tabmlp1983,tabmlp550,lgb1701,xgb1701,cat1701,tablmp1701,lgb2063,xgb2063,cat2063,tablmp2063
0,0.603463,0.603793,0.609108,0.643063,0.644761,0.651015,0.627152,0.666962,0.658337,0.633626,...,0.551580,0.455730,0.647845,0.636991,0.604513,0.550344,0.636336,0.652631,0.630627,0.550438
1,0.999806,0.996125,0.988117,0.979158,0.980269,0.979628,0.975591,0.976313,0.975725,0.978221,...,0.958873,0.965270,0.979586,0.975426,0.977638,0.967344,0.979467,0.973305,0.977416,0.966318
2,0.678293,0.667067,0.679545,0.694800,0.735218,0.685842,0.674095,0.670068,0.668585,0.758380,...,0.600854,0.755067,0.716650,0.679206,0.788293,0.634997,0.642301,0.663404,0.696574,0.747513
3,0.312112,0.317473,0.317336,0.315001,0.310552,0.317150,0.294775,0.321891,0.326354,0.277934,...,0.214472,0.273882,0.301760,0.318009,0.272723,0.223079,0.311523,0.316786,0.280914,0.174859
4,0.028643,0.029519,0.038828,0.086256,0.089822,0.081587,0.069561,0.071851,0.072671,0.060639,...,0.034382,0.082108,0.094915,0.065146,0.066420,0.050387,0.084422,0.068258,0.064579,0.073367


In [101]:
# test_lv1 = pd.DataFrame(test_lv1)

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


In [102]:
test_lv1.head()

,lgb42,lgb1983,lgb550,xgb42,xgb1983,xgb550,cat42,cat1983,cat550,tabmlp42,tabmlp1983,tabmlp550,lgb1701,xgb1701,cat1701,tablmp1701,lgb2063,xgb2063,cat2063,tablmp2063
0,0.704171,0.719313,0.716673,0.742503,0.744670,0.743342,0.731599,0.743098,0.736512,0.702985,0.698789,0.703244,0.709136,0.745753,0.732615,0.641977,0.712405,0.741647,0.729088,0.659293
1,0.225994,0.234714,0.240386,0.263515,0.255078,0.254107,0.239633,0.236084,0.238824,0.300979,0.257073,0.312073,0.240015,0.256124,0.236380,0.248044,0.229450,0.256039,0.242159,0.317020
2,0.905121,0.908202,0.902443,0.909820,0.903481,0.903708,0.909699,0.908522,0.910965,0.849669,0.873373,0.836253,0.907301,0.903944,0.911547,0.855545,0.902819,0.905785,0.907301,0.858448
3,0.803894,0.808522,0.810213,0.861400,0.860360,0.863230,0.847200,0.844481,0.832912,0.857812,0.836521,0.854306,0.807577,0.858694,0.837866,0.869585,0.816616,0.866747,0.836606,0.805957
4,0.282883,0.289195,0.278769,0.261821,0.262067,0.265357,0.266116,0.267789,0.264228,0.209983,0.293265,0.309422,0.281437,0.275409,0.266691,0.231212,0.277253,0.260022,0.265275,0.268644


In [103]:
test_lv2_full = test_lv2.join(test_lv1)

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


In [104]:
test_lv2_full.head()

,xgboost,catboost,lightgbm,lgb42,lgb1983,lgb550,xgb42,xgb1983,xgb550,cat42,...,tabmlp1983,tabmlp550,lgb1701,xgb1701,cat1701,tablmp1701,lgb2063,xgb2063,cat2063,tablmp2063
0,0.696256,0.690326,0.692617,0.704171,0.719313,0.716673,0.742503,0.744670,0.743342,0.731599,...,0.698789,0.703244,0.709136,0.745753,0.732615,0.641977,0.712405,0.741647,0.729088,0.659293
1,0.269481,0.269859,0.275947,0.225994,0.234714,0.240386,0.263515,0.255078,0.254107,0.239633,...,0.257073,0.312073,0.240015,0.256124,0.236380,0.248044,0.229450,0.256039,0.242159,0.317020
2,0.915829,0.912697,0.903958,0.905121,0.908202,0.902443,0.909820,0.903481,0.903708,0.909699,...,0.873373,0.836253,0.907301,0.903944,0.911547,0.855545,0.902819,0.905785,0.907301,0.858448
3,0.807240,0.812514,0.791061,0.803894,0.808522,0.810213,0.861400,0.860360,0.863230,0.847200,...,0.836521,0.854306,0.807577,0.858694,0.837866,0.869585,0.816616,0.866747,0.836606,0.805957
4,0.296900,0.293652,0.302775,0.282883,0.289195,0.278769,0.261821,0.262067,0.265357,0.266116,...,0.293265,0.309422,0.281437,0.275409,0.266691,0.231212,0.277253,0.260022,0.265275,0.268644


In [105]:
oof_lv2_full.head()

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


,xgboost,catboost,lightgbm,lgb42,lgb1983,lgb550,xgb42,xgb1983,xgb550,cat42,...,tabmlp1983,tabmlp550,lgb1701,xgb1701,cat1701,tablmp1701,lgb2063,xgb2063,cat2063,tablmp2063
0,0.603463,0.603793,0.609108,0.643063,0.644761,0.651015,0.627152,0.666962,0.658337,0.633626,...,0.551580,0.455730,0.647845,0.636991,0.604513,0.550344,0.636336,0.652631,0.630627,0.550438
1,0.999806,0.996125,0.988117,0.979158,0.980269,0.979628,0.975591,0.976313,0.975725,0.978221,...,0.958873,0.965270,0.979586,0.975426,0.977638,0.967344,0.979467,0.973305,0.977416,0.966318
2,0.678293,0.667067,0.679545,0.694800,0.735218,0.685842,0.674095,0.670068,0.668585,0.758380,...,0.600854,0.755067,0.716650,0.679206,0.788293,0.634997,0.642301,0.663404,0.696574,0.747513
3,0.312112,0.317473,0.317336,0.315001,0.310552,0.317150,0.294775,0.321891,0.326354,0.277934,...,0.214472,0.273882,0.301760,0.318009,0.272723,0.223079,0.311523,0.316786,0.280914,0.174859
4,0.028643,0.029519,0.038828,0.086256,0.089822,0.081587,0.069561,0.071851,0.072671,0.060639,...,0.034382,0.082108,0.094915,0.065146,0.066420,0.050387,0.084422,0.068258,0.064579,0.073367


In [106]:
oof_lv2_np = oof_lv2_full.to_numpy()
test_lv2_np = test_lv2_full.to_numpy()

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


In [107]:
type(oof_y)

pandas.core.series.Series

In [108]:
oof_y_np = oof_y

## Level Three (Logistic Regression)

In [109]:
from sklearn import model_selection

In [110]:
# kfolds = model_selection.StratifiedKFold(n_splits=5, shuffle=False) # no random_state if shuffle == False

In [111]:
oof_preds, oof_y = [], []

In [112]:
test_preds = np.zeros((X_test.shape[0]))

In [113]:
X = oof_lv2_np
y = oof_y_np
X_test = test_lv2_np

In [114]:
X[:10]

array([[0.60346347, 0.60379302, 0.60910751, 0.6430626 , 0.64476147,
        0.65101507, 0.62715232, 0.66696191, 0.658337  , 0.63362641,
        0.63786275, 0.61812448, 0.52831805, 0.55158025, 0.45572963,
        0.64784501, 0.63699114, 0.60451306, 0.55034405, 0.63633613,
        0.65263098, 0.6306274 , 0.55043793],
       [0.99980623, 0.99612459, 0.98811665, 0.97915809, 0.98026903,
        0.97962814, 0.9755913 , 0.97631311, 0.97572541, 0.97822092,
        0.9788715 , 0.97970244, 0.96083504, 0.95887262, 0.96526986,
        0.97958613, 0.97542602, 0.97763838, 0.96734434, 0.97946677,
        0.97330451, 0.97741563, 0.96631795],
       [0.67829287, 0.66706652, 0.67954529, 0.69480043, 0.73521753,
        0.68584185, 0.6740948 , 0.67006797, 0.66858542, 0.75837971,
        0.69574226, 0.68765168, 0.72254694, 0.60085428, 0.75506681,
        0.71664953, 0.67920649, 0.78829268, 0.63499689, 0.64230075,
        0.66340375, 0.69657356, 0.74751294],
       [0.31211236, 0.31747341, 0.31733638, 0.315

In [115]:
library = 'sklearn (LogisticRegressor(max_iter=1000))'
# exmodel_config['library'] = library
# wandb.init(
#     project="202110_Kaggle_tabular_playground",
#     save_code=True,
#     tags=wandb_config['tags'],
#     name=wandb_config['name'],
#     notes=wandb_config['notes'],
#     config=exmodel_config
# )   

# prepare for k-fold cross-validation
# kfold = exmodel_config['cross_val_strategy'](n_splits=exmodel_config['kfolds'], shuffle=True, random_state=SEED)
kfold = exmodel_config['cross_val_strategy'](n_splits=exmodel_config['kfolds'], shuffle=False)

# setup for serialization
# model_path = Path(datapath/f"models/{wandb_config['name']}_{library}_{exmodel_config['kfolds']}folds/")
# (model_path).mkdir(exist_ok=True)

In [116]:
for fold, (train_idx, valid_idx) in enumerate(kfold.split(X,y)):
    X_train, X_valid = X[train_idx], X[valid_idx]
    y_train, y_valid = y[train_idx], y[valid_idx]
    
    print(f"FOLD {fold}")
    print("---------------------")
    
    model = LogisticRegression(max_iter=1000)
    
    model.fit(X_train, y_train)
    preds = model.predict_proba(X_valid)[:,1]
    
    oof_preds.extend(preds)
    oof_y.extend(y_valid)
    
    test_preds += model.predict_proba(X_test)[:,1]
    
    valid_auc = roc_auc_score(y_valid, preds)
    print(f"ROC AUC of fold {fold} is {valid_auc}")
    
#     dump(preds, /'lv_3)

valid_auc_total = roc_auc_score(oof_y, oof_preds)
print(f"Overall ROC_AUC is {valid_auc_total}")

dump(oof_preds, predpath/f"{wandb_config['name']}nb-{datetime.now().strftime('%Y%m%d%H%M%S')}run-X_orig_oof_lv3_preds.joblib")
dump(oof_y, predpath/'oof_lv3_y.joblib')

test_preds /= 5

dump(test_preds, predpath/f"{wandb_config['name']}nb-{datetime.now().strftime('%Y%m%d%H%M%S')}run-X_orig_test_lv3_preds.joblib")

FOLD 0
---------------------
ROC AUC of fold 0 is 0.8569595004826156
FOLD 1
---------------------
ROC AUC of fold 1 is 0.8558408474590921
FOLD 2
---------------------
ROC AUC of fold 2 is 0.8568110547897336
FOLD 3
---------------------
ROC AUC of fold 3 is 0.8553678789830179
FOLD 4
---------------------
ROC AUC of fold 4 is 0.8565676032821483
Overall ROC_AUC is 0.8562392566675165


['/home/sf/code/kaggle/tabular_playgrounds/oct2021/preds/stacking_manual_20211031_160241nb-20211031165953run-X_orig_test_lv3_preds.joblib']

In [117]:
# X_test_imputed_scaled = pd.read_feather(path=datapath/'X_test_NaNcounts_imputed-Median-wIndicators_StandardScaled.feather')
# X_test_imputed_scaled = pd.read_feather(path=datapath/'X_test_NaNcounts_SummaryStats_imputed-Median-wIndicators-StandardScaled.feather')

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


## Prediction Generation

In [118]:
# preds_path = Path(datapath/"preds/")

# blender_preds = blender.predict_proba(X_test_imputed_scaled)[:,1]
# dump(blender_preds, preds_path/f"{config_run['name']}_stack.joblib")

# Ensemble Submission

In [119]:
# widedeep_preds = load(predpath/'stacking_manual_20211012_194716_widedeep-TabMLP_5folds_rs1983_500epochs_test_preds.joblib')

In [120]:
# widedeep_preds = pd.Series(widedeep_preds, name='widedeep_tabmlp1983')

In [ ]:
# test_lv1_xgb42 = load(predpath/'stacking_manual_20211020_104938_xgboost_5folds_rs42_test_preds.joblib')

In [ ]:
# test_lv1_lgb42 = load(predpath/'stacking_manual_20211020_104938_lightgbm_5folds_rs42_test_preds.joblib')
# test_lv1_cat42 = load(predpath/'stacking_manual_20211020_104938_catboost_5folds_rs42_test_preds.joblib')

In [ ]:
type(test_lv1_xgb42)

In [ ]:
mod_mix_three = 0.5*test_preds + 0.3*test_lv1_xgb42 + 0.05*test_lv1_lgb42 + 0.05*test_lv1_cat42 + 0.1*test_lv1_tabmlp42

In [ ]:
mod_mix_three_pow4 = 0.5*test_preds**4 + 0.3*test_lv1_xgb42**4 + 0.05*test_lv1_lgb42**4 + 0.05*test_lv1_cat42**4 + 0.1*test_lv1_tabmlp42**4

In [ ]:
sample_df = pd.read_csv(datapath/'sample_submission.csv.zip')

In [ ]:
sample_df.loc[:, 'target'] = mod_mix_three_pow4

In [ ]:
sample_df.head()

In [ ]:
# submission_path = datapath/'submissions'
# submission_path.mkdir(exist_ok=True)

In [ ]:
sample_df.to_csv(subpath/f"{wandb_config['name']}_3level_3rs-X_orig-GBM-0.5stack_0.3xgb42_0.1tabmlp42_0.05lgb42_0.05cat42_pow4_ensemble_preds.csv", index=False)
# sample_df.to_csv(subpath/f"{wandb_config['name']}_3level-X_orig+KMeans8+synth-GBM-stack_ensemble_preds.csv", index=False)

## Level 4 (Passthrough)

### Experimentation

In [ ]:
# oof_lv1.head()

Let's see what we can get if we effectively do a pass-through -- that is, we join the results from the first level with the results from the second, and even potentially the third, then feed the resulting table through a few different models. (Given that we'll have a greater number of features, we might want to try XGBoost rather than just a simple logistic regression.

In [121]:
oof_lv1_and_2 = oof_lv1.join(oof_lv2)
oof_lv1_and_2.head()

,lgb42,lgb1983,lgb550,xgb42,xgb1983,xgb550,cat42,cat1983,cat550,tabmlp42,...,xgb1701,cat1701,tablmp1701,lgb2063,xgb2063,cat2063,tablmp2063,xgboost,catboost,lightgbm
0,0.643063,0.644761,0.651015,0.627152,0.666962,0.658337,0.633626,0.637863,0.618124,0.528318,...,0.636991,0.604513,0.550344,0.636336,0.652631,0.630627,0.550438,0.603463,0.603793,0.609108
1,0.979158,0.980269,0.979628,0.975591,0.976313,0.975725,0.978221,0.978872,0.979702,0.960835,...,0.975426,0.977638,0.967344,0.979467,0.973305,0.977416,0.966318,0.999806,0.996125,0.988117
2,0.694800,0.735218,0.685842,0.674095,0.670068,0.668585,0.758380,0.695742,0.687652,0.722547,...,0.679206,0.788293,0.634997,0.642301,0.663404,0.696574,0.747513,0.678293,0.667067,0.679545
3,0.315001,0.310552,0.317150,0.294775,0.321891,0.326354,0.277934,0.281502,0.280313,0.200837,...,0.318009,0.272723,0.223079,0.311523,0.316786,0.280914,0.174859,0.312112,0.317473,0.317336
4,0.086256,0.089822,0.081587,0.069561,0.071851,0.072671,0.060639,0.061077,0.068461,0.060948,...,0.065146,0.066420,0.050387,0.084422,0.068258,0.064579,0.073367,0.028643,0.029519,0.038828


In [122]:
oof_all_lvs = oof_lv1_and_2.join(pd.Series(oof_preds, name='lv3_logistic_reg_preds'))
oof_all_lvs

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


,lgb42,lgb1983,lgb550,xgb42,xgb1983,xgb550,cat42,cat1983,cat550,tabmlp42,...,cat1701,tablmp1701,lgb2063,xgb2063,cat2063,tablmp2063,xgboost,catboost,lightgbm,lv3_logistic_reg_preds
0,0.643063,0.644761,0.651015,0.627152,0.666962,0.658337,0.633626,0.637863,0.618124,0.528318,...,0.604513,0.550344,0.636336,0.652631,0.630627,0.550438,0.603463,0.603793,0.609108,0.620127
1,0.979158,0.980269,0.979628,0.975591,0.976313,0.975725,0.978221,0.978872,0.979702,0.960835,...,0.977638,0.967344,0.979467,0.973305,0.977416,0.966318,0.999806,0.996125,0.988117,0.967607
2,0.694800,0.735218,0.685842,0.674095,0.670068,0.668585,0.758380,0.695742,0.687652,0.722547,...,0.788293,0.634997,0.642301,0.663404,0.696574,0.747513,0.678293,0.667067,0.679545,0.695375
3,0.315001,0.310552,0.317150,0.294775,0.321891,0.326354,0.277934,0.281502,0.280313,0.200837,...,0.272723,0.223079,0.311523,0.316786,0.280914,0.174859,0.312112,0.317473,0.317336,0.320700
4,0.086256,0.089822,0.081587,0.069561,0.071851,0.072671,0.060639,0.061077,0.068461,0.060948,...,0.066420,0.050387,0.084422,0.068258,0.064579,0.073367,0.028643,0.029519,0.038828,0.033848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,0.877003,0.866812,0.859368,0.873962,0.876338,0.882213,0.876955,0.876530,0.873538,0.872165,...,0.875290,0.880338,0.863979,0.866435,0.880339,0.894021,0.858877,0.856541,0.838445,0.872278
999996,0.735823,0.732218,0.702689,0.777820,0.785770,0.789151,0.766000,0.766401,0.768164,0.751403,...,0.768249,0.743479,0.712077,0.768061,0.726242,0.661662,0.721611,0.721759,0.722971,0.756245
999997,0.151834,0.155176,0.146293,0.140220,0.146466,0.153901,0.164341,0.165027,0.164354,0.103621,...,0.155469,0.077828,0.143994,0.135747,0.161125,0.168268,0.169669,0.166342,0.183475,0.159699
999998,0.205738,0.182986,0.196857,0.216337,0.200384,0.197216,0.232795,0.227559,0.229330,0.277918,...,0.229789,0.335021,0.186798,0.224917,0.226820,0.178284,0.251595,0.252851,0.254263,0.248749


In [123]:
test_all_lvs = test_lv1.join(test_lv2).join(pd.Series(test_preds, name='lv3_logistic_test_preds'))

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


In [124]:
test_all_lvs.head()

,lgb42,lgb1983,lgb550,xgb42,xgb1983,xgb550,cat42,cat1983,cat550,tabmlp42,...,cat1701,tablmp1701,lgb2063,xgb2063,cat2063,tablmp2063,xgboost,catboost,lightgbm,lv3_logistic_test_preds
0,0.704171,0.719313,0.716673,0.742503,0.744670,0.743342,0.731599,0.743098,0.736512,0.702985,...,0.732615,0.641977,0.712405,0.741647,0.729088,0.659293,0.696256,0.690326,0.692617,0.701891
1,0.225994,0.234714,0.240386,0.263515,0.255078,0.254107,0.239633,0.236084,0.238824,0.300979,...,0.236380,0.248044,0.229450,0.256039,0.242159,0.317020,0.269481,0.269859,0.275947,0.267346
2,0.905121,0.908202,0.902443,0.909820,0.903481,0.903708,0.909699,0.908522,0.910965,0.849669,...,0.911547,0.855545,0.902819,0.905785,0.907301,0.858448,0.915829,0.912697,0.903958,0.938763
3,0.803894,0.808522,0.810213,0.861400,0.860360,0.863230,0.847200,0.844481,0.832912,0.857812,...,0.837866,0.869585,0.816616,0.866747,0.836606,0.805957,0.807240,0.812514,0.791061,0.815654
4,0.282883,0.289195,0.278769,0.261821,0.262067,0.265357,0.266116,0.267789,0.264228,0.209983,...,0.266691,0.231212,0.277253,0.260022,0.265275,0.268644,0.296900,0.293652,0.302775,0.308340


In [ ]:
oof_all_lvs.to_feather(altdatapath/'oof_all_lvs_passthru_20211031-final.feather')
test_all_lvs.to_feather(altdatapath/'test_all_lvs_passthru_20211031-final.feather')

In [ ]:
xgb_lv4_params = {
    'n_estimators': 3878,
    'max_depth': 4,
    'learning_rate': 0.024785857161974977,
    'reg_alpha': 26.867682044658245,
    'reg_lambda': 10.839759074147148,
    'subsample': 0.8208581489835881,
    'min_child_weight': 8.829122644339664,
    'colsample_bytree': 0.906420714280384,
    'gamma': 1.472322916021486
}

In [72]:
xgb_lv4_model = XGBClassifier(
    objective='binary:logistic',
    verbosity=1,
    tree_method='gpu_hist',
    booster='gbtree', # not bothering with dart for time reasons
    random_state=SEED,
    **xgb_lv4_params
)

In [73]:
X_lv4_train, X_lv4_valid, y_lv4_train, y_lv4_valid = train_test_split(oof_all_lvs, oof_y, test_size=0.2, random_state=int(SEED), shuffle=True)

In [74]:
xgb_lv4_model.fit(X_lv4_train, y_lv4_train)

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[16:15:44] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.906420714280384,
              gamma=1.472322916021486, gpu_id=0, importance_type='gain',
              interaction_constraints='', learning_rate=0.024785857161974977,
              max_delta_step=0, max_depth=4, min_child_weight=8.829122644339664,
              missing=nan, monotone_constraints='()', n_estimators=3878,
              n_jobs=16, num_parallel_tree=1, random_state=42,
              reg_alpha=26.867682044658245, reg_lambda=10.839759074147148,
              scale_pos_weight=1, subsample=0.8208581489835881,
              tree_method='gpu_hist', validate_parameters=1, verbosity=1)

In [75]:
lv4_train_preds = xgb_lv4_model.predict_proba(X_lv4_valid)[:,1]

In [76]:
roc_auc_score(y_score=lv4_train_preds, y_true=y_lv4_valid)

0.8568572581392072

Let's compare that number to what we'd gotten at lv 3, and then what we'd get at lv 4 with LogisticRegression

In [77]:
log_lv4_model = LogisticRegression(max_iter=1000)

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


In [78]:
log_lv4_model.fit(X_lv4_train, y_lv4_train)

LogisticRegression(max_iter=1000)

In [79]:
lv4_log_train_preds = log_lv4_model.predict_proba(X_lv4_valid)[:,1]

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


In [80]:
roc_auc_score(y_score=lv4_log_train_preds, y_true=y_lv4_valid)

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


0.8568131691662475

So XGBoost at this level does do a bit better than LogisticRegression would. But should it come to this point? Let's find out.

### Adding more models


### Run

In [ ]:
oof_all_lvs = pd.read_feather(altdatapath/'oof_all_lvs_passthru_202110300859.feather')
test_all_lvs = pd.read_feather(altdatapath/'test_all_lvs_passthru_202110300859.feather')

In [126]:
y = oof_y_np
test_preds = np.zeros((X_test.shape[0]))

In [127]:
# y = oof_y
oof_preds = []
oof_y = []

for fold, (train_idx, valid_idx) in enumerate(kfold.split(oof_all_lvs, y)):
    X_train, X_valid = X[train_idx], X[valid_idx]
    y_train, y_valid = y[train_idx], y[valid_idx]
    
    print(f"FOLD {fold}")
    print("---------------------")
    
    model = XGBClassifier(
        objective='binary:logistic',
        verbosity=1,
        tree_method='gpu_hist',
        booster='gbtree', # not bothering with dart for time reasons
        random_state=SEED,
        **xgb_lv4_params
    )
    model.fit(X_train, y_train)
    preds = model.predict_proba(X_valid)[:,1]
    
    oof_preds.extend(preds)
    oof_y.extend(y_valid)
    
    test_preds += model.predict_proba(X_test)[:,1]
    
    valid_auc = roc_auc_score(y_valid, preds)
    print(f"ROC AUC of fold {fold} is {valid_auc}")
    
#     dump(preds, /'lv_3)

valid_auc_total = roc_auc_score(y_true=oof_y, y_score=oof_preds)
print(f"Overall ROC_AUC is {valid_auc_total}")

dump(oof_preds, predpath/f"{wandb_config['name']}nb-{datetime.now().strftime('%Y%m%d%H%M%S')}run-X_orig_oof_lv4xgb_passthru_preds.joblib")
dump(oof_y, predpath/'oof_lv4_y.joblib')

test_preds /= 5

dump(test_preds, predpath/f"{wandb_config['name']}nb-{datetime.now().strftime('%Y%m%d%H%M%S')}run-X_orig_test_lv4xgb_passthru_preds.joblib")

FOLD 0
---------------------
[17:00:07] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
ROC AUC of fold 0 is 0.8576973814950857
FOLD 1
---------------------


The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[17:00:25] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
ROC AUC of fold 1 is 0.8564821246032219
FOLD 2
---------------------


The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[17:00:44] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
ROC AUC of fold 2 is 0.8574951345643358
FOLD 3
---------------------


The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[17:01:03] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
ROC AUC of fold 3 is 0.8563175511608968
FOLD 4
---------------------


The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[17:01:21] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
ROC AUC of fold 4 is 0.8570563164588454
Overall ROC_AUC is 0.8569724079813386


['/home/sf/code/kaggle/tabular_playgrounds/oct2021/preds/stacking_manual_20211031_160241nb-20211031170152run-X_orig_test_lv4xgb_passthru_preds.joblib']

Notionally better.

In [ ]:
test_preds[:10]

--------------

In [128]:
sample_df = pd.read_csv(datapath/'sample_submission.csv.zip')

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


In [129]:
sample_df.loc[:, 'target'] = test_preds

In [130]:
sample_df.head()

,id,target
0,1000000,0.708235
1,1000001,0.262932
2,1000002,0.916396
3,1000003,0.808739
4,1000004,0.284263


In [131]:
# submission_path = datapath/'submissions'
# submission_path.mkdir(exist_ok=True)

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.


In [132]:
sample_df.to_csv(subpath/f"{wandb_config['name']}_4level-GBM+TabMLP-ensemble_X-orig_3rs_{exmodel_config['kfolds']}folds_finalrs{42}_preds-FINAL.csv", index=False)

LB 0.85628 -- an improvement, at long last.

In [ ]:
# wandb.log({'leaderboard_auc': ,
# #            'catboost_params': str(best_catboost_params),
#           })

In [ ]:
# wandb.finish()

# Power Averaging

In [ ]:
stack_sub = sample_df.copy()

In [ ]:
stack_preds = stack_sub.iloc[:,1]

In [ ]:
# cat1983_preds = load(predpath/'stacking_manual_20211005_205933_catboost_5folds_rs1983_test_preds.joblib')

In [ ]:
# cat_preds[:10]

In [ ]:
# stack_sub.iloc[:10,1]

In [ ]:
# lgb1983_preds = load(predpath/'stacking_manual_20211005_205933_lightgbm_5folds_rs1983_test_preds.joblib')

In [ ]:
# lgb1983_preds[:10]

## Correlations
Before I make these predictions, let's try some rigor.

In [ ]:
type(test_lv1)

In [ ]:
test_lv1.shape

In [ ]:
sns.heatmap(test_lv1.iloc[:,:].corr(), annot=True)
plt.show()

- So what this is saying is,XGBoost (models 0 and 1) is a bit closer to LightGBM (models 2 and 3) than to CatBoost (models 4 and 5), and LightGBM is closer to Catboost than to XGBoost.

What if we bring in the full stack's predictions?

In [ ]:
stack_feature = pd.Series(test_preds, name='full_stack')
corr_compare = test_lv1.join(stack_feature)
corr_compare.head()

In [ ]:
sns.heatmap(corr_compare.corr(), annot=True)
plt.show()

- So this is indicating that the stack is further apart from each of the GBMs than they are from one another -- but, it's marginally closer to CatBoost than to LightGBM, and it's definitely closer to both of those than to XGBoost alone. **So, wrt power averaging, it actually may not make sense to power average with CatBoost and LightGBM alongside the stack.** 

What if we bring in some `widedeep` predictions too?

In [ ]:
widedeep_preds = load(predpath/'stacking_manual_20211012_194716_widedeep-TabMLP_5folds_rs1983_500epochs_test_preds.joblib')

In [ ]:
type(widedeep_preds)

In [ ]:
widedeep_preds = pd.Series(widedeep_preds, name='widedeep_tabmlp1983')

In [ ]:
corr_compare_deep = corr_compare.join(widedeep_preds)

In [ ]:
corr_compare_deep.head()

In [ ]:
sns.heatmap(corr_compare_deep.corr(), annot=True)
plt.show()

## Modified Mix

In [ ]:
mod_mix = 0.5*stack_preds + 0.4*test_lv1_xgb42 + 0.1*widedeep_preds

In [ ]:
mod_mix[:10]

In [ ]:
sample_df = pd.read_csv(datapath/'sample_submission.csv.zip')

In [ ]:
sample_df.loc[:, 'target'] = mod_mix

In [ ]:
sample_df.head()

In [ ]:
# submission_path = datapath/'submissions'
# submission_path.mkdir(exist_ok=True)

In [ ]:
sample_df.to_csv(subpath/f"{wandb_config['name']}_3level-GBM-X_orig+KMeans8+synth-0.5stack_0.4xgb42_0.1tabmlp1983_ensemble_preds.csv", index=False)

## Mod Mix Two

In [ ]:
mod_mix_two = 0.6*stack_preds + 0.2*test_lv1_xgb42 + 0.05*test_lv1_lgb42 + 0.05*test_lv1_cat42 + 0.1*widedeep_preds

In [ ]:
mod_mix_two[:10]

In [ ]:
sample_df = pd.read_csv(datapath/'sample_submission.csv.zip')

In [ ]:
sample_df.loc[:, 'target'] = mod_mix_two

In [ ]:
sample_df.head()

In [ ]:
# submission_path = datapath/'submissions'
# submission_path.mkdir(exist_ok=True)

In [ ]:
sample_df.to_csv(subpath/f"{wandb_config['name']}_3level-X_orig+KMeans8+synth-GBM-0.6stack_0.2xgb42_0.1tabmlp1983_0.05lgb42_0.05cat42_ensemble_preds.csv", index=False)

# Mod Mix Three

In [ ]:
mod_mix_three = 0.5*stack_preds + 0.3*test_lv1_xgb42 + 0.05*test_lv1_lgb42 + 0.05*test_lv1_cat42 + 0.1*widedeep_preds

In [ ]:
sample_df = pd.read_csv(datapath/'sample_submission.csv.zip')

In [ ]:
sample_df.loc[:, 'target'] = mod_mix_three

In [ ]:
sample_df.head()

In [ ]:
# submission_path = datapath/'submissions'
# submission_path.mkdir(exist_ok=True)

In [ ]:
sample_df.to_csv(subpath/f"{wandb_config['name']}_3level-X_orig+KMeans8+synth-GBM-0.5stack_0.3xgb42_0.1tabmlp1983_0.05lgb42_0.05cat42_ensemble_preds.csv", index=False)

## 4th power

In [ ]:
mod_mix_three_pow4 = 0.5*stack_preds**4 + 0.3*test_lv1_xgb42**4 + 0.05*test_lv1_lgb42**4 + 0.05*test_lv1_cat42**4 + 0.1*widedeep_preds**4

In [ ]:
sample_df = pd.read_csv(datapath/'sample_submission.csv.zip')

In [ ]:
sample_df.loc[:, 'target'] = mod_mix_three_pow4

In [ ]:
sample_df.head()

In [ ]:
# submission_path = datapath/'submissions'
# submission_path.mkdir(exist_ok=True)

In [ ]:
sample_df.to_csv(subpath/f"{wandb_config['name']}_3level-X_orig+KMeans8+synth-GBM-0.5stack_0.3xgb42_0.1tabmlp1983_0.05lgb42_0.05cat42_pow4_ensemble_preds.csv", index=False)

In [ ]:
power4_avg_alt = 0.5*stack_preds**4 + 0.4*test_lv1_xgb42 + 0.1*widedeep_preds** 4

In [ ]:
power4_avg_alt[:20]

In [ ]:
sample_df = pd.read_csv(datapath/'sample_submission.csv.zip')

In [ ]:
sample_df.loc[:, 'target'] = power4_avg_alt

In [ ]:
sample_df.head()

In [ ]:
# submission_path = datapath/'submissions'
# submission_path.mkdir(exist_ok=True)

In [ ]:
sample_df.to_csv(subpath/f"{wandb_config['name']}_3level-GBM-0.5stack_0.4xgb42_0.1tabmlp1983_pow-avg4_ensemble_preds.csv", index=False)

## 6th power

In [ ]:
power6_avg = (stack_preds**6 + cat1983_preds**6 + lgb1983_preds**6) / 3

In [ ]:
power6_avg[:20]

In [ ]:
sample_df = pd.read_csv(datapath/'sample_submission.csv.zip')

In [ ]:
sample_df.loc[:, 'target'] = power6_avg

In [ ]:
sample_df.head()

In [ ]:
# submission_path = datapath/'submissions'
# submission_path.mkdir(exist_ok=True)

In [ ]:
sample_df.to_csv(subpath/f"{wandb_config['name']}_3level-GBM-stack_cat1983_lgb1983_pow-avg6_ensemble_preds.csv", index=False)

## 5th power

In [ ]:
power5_avg = (stack_preds**5 + cat1983_preds**5 + lgb1983_preds**5) / 3

In [ ]:
# power5_avg[:20]

In [ ]:
sample_df = pd.read_csv(datapath/'sample_submission.csv.zip')

In [ ]:
sample_df.loc[:, 'target'] = power5_avg

In [ ]:
sample_df.head()

In [ ]:
# submission_path = datapath/'submissions'
# submission_path.mkdir(exist_ok=True)

In [ ]:
sample_df.to_csv(subpath/f"{wandb_config['name']}_3level-GBM-stack_cat1983_lgb1983_pow-avg5_ensemble_preds.csv", index=False)

## 3rd power

In [ ]:
power3_avg = (stack_preds**3 + cat1983_preds**3 + lgb1983_preds**3) / 3

In [ ]:
# power5_avg[:20]

In [ ]:
sample_df = pd.read_csv(datapath/'sample_submission.csv.zip')

In [ ]:
sample_df.loc[:, 'target'] = power3_avg

In [ ]:
sample_df.head()

In [ ]:
# submission_path = datapath/'submissions'
# submission_path.mkdir(exist_ok=True)

In [ ]:
sample_df.to_csv(subpath/f"{wandb_config['name']}_3level-GBM-stack_cat1983_lgb1983_pow-avg3_ensemble_preds.csv", index=False)